In [50]:
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
import tensorflow as tf

label_to_num = {'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'optimism': 20,
 'pride': 21,
 'realization': 22,
 'relief': 23,
 'remorse': 24,
 'sadness': 25,
 'surprise': 26,
 'neutral': 27}

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=28)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
model_path = './checkpoints'
checkpoint = tf.train.Checkpoint(model=model)
latest_checkpoint = tf.train.latest_checkpoint(model_path)
checkpoint.restore(latest_checkpoint)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
if status.assert_existing_objects_matched():
    print("Model restored successfully!")
else:
    print("Model not restored!")

Model restored successfully!


In [54]:
model = tf.keras.models.load_model(
    './model_final',
    custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification}
)

In [52]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model_path = './checkpoints'
checkpoint = tf.train.Checkpoint(model=model)
latest_checkpoint = tf.train.latest_checkpoint(model_path)
checkpoint.restore(latest_checkpoint)

tf.keras.models.save_model(
    model,
    './model_final',
    save_format='tf',
    include_optimizer=True
)


INFO:tensorflow:Assets written to: ./model_final/assets


INFO:tensorflow:Assets written to: ./model_final/assets


In [55]:
model.summary()

Model: "tf_bert_for_sequence_classification_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_341 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  21532     
                                                                 
Total params: 109,503,772
Trainable params: 109,503,772
Non-trainable params: 0
_________________________________________________________________


In [56]:
def prediction(pred_sentences):
    
    tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    label = tf.argmax(tf_predictions, axis=1)
    label= label.numpy()
    key_list = list(label_to_num.keys())
    val_list = list(label_to_num.values())

    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    for i in range(len(pred_sentences)):
        position = val_list.index(label[i])
        return key_list[position]

In [57]:
pred_sentences = ["What an excellent sequel - I, in fact, like it more than its predecessor.'Top Gun: Maverick' is fantastic, simply put. I was expecting it to be good, but it's actually much more enjoyable than I had anticipated. The callbacks to the original are expertly done, the new characters are strong/well cast, it has plenty of meaning, music is fab and the action is outstanding - the aerial stuff is sensational. The story is superb, with each high stake coming across as intended - parts even gave me slight goosebumps, which is a surprise given I'm not someone who has a connection to the 1986 film. It's all super neatly put together, I honestly came close to giving it a higher rating. Tom Cruise is brilliant as he reprises the role of Maverick, while Miles Teller comes in and gives a top performance. Jennifer Connelly is another positive, though her role does kinda feel a tiny bit forced in order to have a love interest; given Kelly McGillis' (unexplained) absence. Monica Barbaro stands out most from the fresh faces, though I actually did enjoy watching them all - which is something I thought the film may struggle with, adding new people, but it's done nicely; sure Jon Hamm and Glen Powell are a little cliché, though overall I approve. A great watch - I'd highly recommend it, though naturally would suggest watching the previous film first if you haven't already."]
prediction(pred_sentences)


'admiration'